In [74]:
import pandas as pd
import requests
import json
import time
from dotenv import load_dotenv
import os

In [75]:
load_dotenv('google.env')

True

In [3]:
directory = pd.read_csv('../data/HD2024.csv')

In [65]:
def search_directory(query):
    query = query.lower()
    term_checker = lambda row: query in row.lower() if type(row) == str else False

    search_results = pd.concat([
        directory[directory['INSTNM'].apply(term_checker)],
        directory[directory['IALIAS'].apply(term_checker)]
    ]).drop_duplicates().reset_index(drop=True)

    return search_results

In [76]:
def param_string(parameters):
    string = ''
    for key in parameters:
        string += '&' + str(key) + '=' + str(parameters[key])
    return string

def assemble_query(parameters):
    url = f'https://customsearch.googleapis.com/customsearch/v1?cx={os.getenv("ENGINE_ID")}&key={os.getenv("API_KEY")}'
    url += param_string(parameters)
    url = url.replace(' ','+')
    return url

def send_query(query):
    data = requests.get(query)
    time.sleep(1)
    if data.status_code == 200:
        data = data.json()
        if 'items' in data:
            results = pd.DataFrame(data['items'])
            results['Query'] = query
            return results
    else:
        raise ValueError(f'query failed with status: {data.status_code}')

In [ ]:
def retrieve_cds(unitid):
    curr_college = directory[directory['UNITID'] == unitid].reset_index(drop=True)

    query = assemble_query({
        'siteSearch': curr_college['WEBADDR'][0],
        'siteSearchFilter': 'i',
        'q': 'common data set',
        'fileType': 'pdf'
    })

    results = send_query(query)

    return results[['htmlTitle', 'link']].to_dict(orient='records')

In [96]:
result = {}

In [97]:
search_results = search_directory('Reed College')

In [98]:
unitid = 209922

In [117]:
retrieve_cds(126562)

[{'htmlTitle': '<b>Common Data Set</b> 2021-2022',
  'link': 'https://www.ucdenver.edu/docs/librariesprovider192/default-document-library/rptcds2022.pdf?sfvrsn=da2198ba_2'},
 {'htmlTitle': 'A. General Information',
  'link': 'https://www.ucdenver.edu/docs/librariesprovider192/default-document-library/cds2022-2023.pdf'},
 {'htmlTitle': '<b>Common Data Set</b> 2024-2025',
  'link': 'https://www.ucdenver.edu/docs/librariesprovider192/default-document-library/cds-24-25.pdf?sfvrsn=fe11edb4_1'},
 {'htmlTitle': '<b>Common Data Set</b> 2020-2021',
  'link': 'https://www.ucdenver.edu/docs/librariesprovider192/default-document-library/rptcds2021.pdf?sfvrsn=7620baba_2'},
 {'htmlTitle': 'A. General Information',
  'link': 'https://www.ucdenver.edu/docs/librariesprovider192/default-document-library/rptcds20217f07fce6302864d9a5bfff0a001ce385.pdf?sfvrsn=b02f98ba_2'},
 {'htmlTitle': 'Admissions Working Group Meeting',
  'link': 'https://www.ucdenver.edu/docs/librariesprovider266/awg/11-13-19-awg-minut